In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot

In [3]:
#new_model = tf.keras.models.load_model("D:/Modelos/EfficientNet_B1_IMAGENET_IC")

In [4]:
#new_model.summary()

In [5]:
#carrega os arrays de imagens e de classes
X = np.load("D:/Displasia/displasiaDataAug/displasiaOriginalDataX_teste3_GLCM.npy")
Y = np.load("D:/Displasia/displasiaDataAug/displasiaOriginalDataYOneHot_teste3.npy")
#dataX = np.load("Image Arrays/displasiaGeneratedDataX.npy")
#dataY = np.load("Image Arrays/displasiaGeneratedDataY.npy")

#shape dos arrays
X.shape, Y.shape

((75776, 24), (75776, 4))

In [6]:
from sklearn.model_selection import train_test_split

#divide os dados em 80% para treino e 20% para teste
train_X, test_X, train_label, test_label = train_test_split(X, Y, test_size=0.2, random_state=13)

#divide os dados de treino em 80% para treino e 20% para validação
#train_X, valid_X, train_label, valid_label = train_test_split(train_X, train_label, test_size=0.2, random_state=13)

#mostra os shapes resultantes das divisões
print("Training:   ", train_X.shape, train_label.shape)
#print("Validation: ", valid_X.shape, valid_label.shape)
print("Testing:    ", test_X.shape, test_label.shape)

Training:    (60620, 24) (60620, 4)
Testing:     (15156, 24) (15156, 4)


In [7]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
#kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
kfold = KFold(n_splits=5, shuffle=True, random_state=7)
cvscores = []
num_kfold=1
#from tensorflow.keras.layers import Activation, Dense
#new_model = GlobalAveragePooling2D()(model)
#drop = Dropout(0.2)(new_model)
#new_model2 = Dense(4,activation='relu')(drop)

#new_model3 = Model(inputs=model.input, outputs=new_model)
#layers.GlobalAveragePooling2D(
#    data_format=None, keepdims=False, #**kwargs
#)

mean = [[[],[],[],[],[]],
         [[],[],[],[],[]],
         [[],[],[],[],[]],
         [[],[],[],[],[]]]

std = [[[],[],[],[],[]],
         [[],[],[],[],[]],
         [[],[],[],[],[]],
         [[],[],[],[],[]]]

for train, test in kfold.split (X, Y):
    
    clf=RandomForestClassifier(n_estimators=100)
    #train_labels2=train_labels[:49984]
    clf.fit(X[train],Y[train])
    
    y_pred=clf.predict(X[test])
    print("Accuracy:",metrics.accuracy_score(Y[test], y_pred))
    #faz a predição das imagens

    from keras.utils import to_categorical

    predicted_classes = y_pred

    predicted_classes = np.argmax(np.round(predicted_classes),axis=1)

    predicted_classes = to_categorical(predicted_classes)

#    predicted_classes.shape, test_label.shape

    num_classes = 4

    target_names = ["Class {}".format(i) for i in range(num_classes)]
    print(classification_report(Y[test], predicted_classes, target_names=target_names))

    #converte os arrays de classe de one-hot para númerico
    test_label = np.argmax(Y[test], axis=1)
    predicted_classes = np.argmax(predicted_classes, axis=1)

    #gera a matriz de confusão a partir da predição feita
    cm = confusion_matrix(test_label, predicted_classes)

    #gera os valores de falso positivo, falso negativo, verdadeiro positivo e verdadeiro negativo
    fp = cm.sum(axis=0) - np.diag(cm)  
    fn = cm.sum(axis=1) - np.diag(cm)
    tp = np.diag(cm)
    tn = cm.sum() - (fp + fn + tp)

    #gera a sensitividade, especificidade, acurácia e precisão
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    accuracy    = (tp + tn) / (tp + tn + fp + fn)
    precision   = tp / (tp + fp)
    f1_score = 2/((1/precision)+(1/sensitivity))

    #mostra os resultados
    print("Precision:  ", precision)
    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)
    print("F1-score:   ", f1_score)
    print("Accuracy:   ", accuracy)
    
    x = []
    x.append(precision)
    x.append(sensitivity)
    x.append(specificity)
    x.append(f1_score)
    x.append(accuracy)
    
    x = np.asarray(x)
    
    x = x.transpose()
    print()
    print(x)
    
    for i in range(4):
        for j in range(5):
            mean[i][j].append(x[i][j])
            std[i][j].append(x[i][j])

Accuracy: 0.8854579044602797
              precision    recall  f1-score   support

     Class 0       0.76      0.99      0.86      3867
     Class 1       0.99      0.89      0.94      3755
     Class 2       0.99      0.89      0.94      3686
     Class 3       0.99      0.88      0.93      3848

   micro avg       0.91      0.91      0.91     15156
   macro avg       0.93      0.91      0.92     15156
weighted avg       0.93      0.91      0.92     15156
 samples avg       0.91      0.91      0.91     15156

Precision:   [0.75576128 0.9928465  0.99153567 0.99180328]
Sensitivity: [0.99224205 0.88708389 0.8898535  0.88045738]
Specificity: [0.89015856 0.99789492 0.99755885 0.99752388]
F1-score:    [0.85800537 0.93699015 0.93794681 0.93281938]
Accuracy:    [0.9162048  0.97044075 0.97136448 0.96780153]

[[0.75576128 0.99224205 0.89015856 0.85800537 0.9162048 ]
 [0.9928465  0.88708389 0.99789492 0.93699015 0.97044075]
 [0.99153567 0.8898535  0.99755885 0.93794681 0.97136448]
 [0.99180328

In [8]:
print(mean)

[[[0.7557612763442978, 0.7558116029917121, 0.7556089743589743, 0.7408840904461194, 0.7497548538929202], [0.9922420480993018, 0.9941504918904547, 0.9921094160967912, 0.9950752393980848, 0.9906711583311739], [0.890158561431482, 0.8939792873442163, 0.892539416894213, 0.8893913043478261, 0.8870396600566572], [0.8580053667262969, 0.8587505741846577, 0.8578576302024108, 0.8493694535263895, 0.8535387363250726], [0.9162048033782001, 0.9188386671065655, 0.9175189706367536, 0.9148795776971297, 0.9134279115803365]], [[0.9928464977645306, 0.9982195845697329, 0.9899259259259259, 0.994174191669094, 0.9946412622804406], [0.8870838881491345, 0.8908898305084746, 0.8799051883065578, 0.8885706847175214, 0.8850331125827815], [0.9978949214981142, 0.9994727128921698, 0.9970065152315548, 0.9982322785928938, 0.9984182776801406], [0.9369901547116738, 0.9415057374755108, 0.9316787506971557, 0.93841077811383, 0.9366414353798711], [0.9704407495381367, 0.9724183437809304, 0.9676674364896074, 0.9704387990762124, 0.

In [25]:
mean = np.asarray(mean)
std = np.asarray(std)
for i in mean:
    print(np.mean(i,axis=1))
print()
for i in std:
    print(np.std(i,axis=1))

[0.75156416 0.99284967 0.89062165 0.85550435 0.91617399]
[0.99396149 0.88629654 0.99820494 0.93704537 0.97022804]
[0.99392212 0.88259909 0.99820748 0.9349414  0.96927784]
[0.98707631 0.8879063  0.9961325  0.93485448 0.96906674]

[0.00581987 0.00156897 0.00242942 0.003573   0.00190404]
[0.00268952 0.00372386 0.00079823 0.00318515 0.00151305]
[0.00166816 0.00651921 0.00046799 0.00322964 0.00187309]
[0.00386571 0.00399747 0.00114012 0.00156156 0.00079378]


In [9]:
#import keras
#intermediate_layer_model = keras.Model(inputs=new_model.input,
#                                       outputs=new_model.get_layer('flatten').output)
#intermediate_output = intermediate_layer_model.predict(valid_X)

In [10]:
#newArr = np.append(intermediate_output, GLCM_features2 , axis=1)

In [11]:
#from sklearn.ensemble import RandomForestClassifier

In [12]:
#clf=RandomForestClassifier(n_estimators=100)
#train_labels2=train_labels[:49984]
#clf.fit(train_X,train_label)

In [13]:
#intermediate_layer_model_2 = keras.Model(inputs=new_model.input,
#                                       outputs=new_model.get_layer('flatten').output)
#intermediate_output_2 = intermediate_layer_model_2.predict(test_X)

In [14]:
#newArr_test = np.append(intermediate_output_2, GLCM_features2_test , axis=1)

In [15]:
#y_pred=clf.predict(test_X)

In [16]:
#from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
#print("Accuracy:",metrics.accuracy_score(test_label, y_pred))

In [17]:
#faz a predição das imagens

#from keras.utils import to_categorical

#predicted_classes = y_pred

#predicted_classes = np.argmax(np.round(predicted_classes),axis=1)

#predicted_classes = to_categorical(predicted_classes)

#predicted_classes.shape, test_label.shape

In [18]:
#mostra as estatísticas de cada classe
#num_classes = 4

#from sklearn.metrics import classification_report
#target_names = ["Class {}".format(i) for i in range(num_classes)]
#print(classification_report(test_label, predicted_classes, target_names=target_names))

In [19]:
#from sklearn.metrics import confusion_matrix

#converte os arrays de classe de one-hot para númerico
#test_label = np.argmax(test_label, axis=1)
#predicted_classes = np.argmax(predicted_classes, axis=1)

#gera a matriz de confusão a partir da predição feita
#cm = confusion_matrix(test_label, predicted_classes)

#gera os valores de falso positivo, falso negativo, verdadeiro positivo e verdadeiro negativo
#fp = cm.sum(axis=0) - np.diag(cm)  
#fn = cm.sum(axis=1) - np.diag(cm)
#tp = np.diag(cm)
#tn = cm.sum() - (fp + fn + tp)

#gera a sensitividade, especificidade, acurácia e precisão
#sensitivity = tp / (tp + fn)
#specificity = tn / (tn + fp)
#accuracy    = (tp + tn) / (tp + tn + fp + fn)
#precision   = tp / (tp + fp)
#f1_score = 2/((1/precision)+(1/sensitivity))

#mostra os resultados
#print("Precision:  ", precision)
#print("Sensitivity:", sensitivity)
#print("Specificity:", specificity)
#print("F1-score:   ", f1_score)
#print("Accuracy:   ", accuracy)